# Расшифровка VIN-кодов автомобилей при помощи регулярных выражений

**Описание проекта**
Изучение информации, зашифрованной в VIN-кодах с целью прогнозирования цены.

**Цель проекта**
- Составление прогноза цен для автомобилей FORD на основании информации скрытой в VIN-кодах.

**Задачи**
- изучить структуру VIN-кода: какие информационные компоненты он содержит (регион, производитель, характеристики автомобиля и т. д.);
- составлять регулярные выражения для поиска и извлечения конкретных частей VIN-кода, таких как производитель, модель и год выпуска;
- распарсить текстовый массив данных с VIN-кодами и ценами;
- решить задачу предсказания цены по VIN-коду.

**Метрики**
- `MSE`, `MAE`, `MAPE`

**Ход исследования**
- Ознакомление с данными;
- ознакомление со способами кодирования данных в VIN-кодах;
- составление паттерна для парсинга VIN-кодов;
- проверка работоспособности паттерна;
- парсинг VIN-кодов;
- подготовка данных для модели;
- обучение baseline модели;
- обучение других моделей;
- выбор лучшей модели;
- проверка важности признаков;
- заключение по исследованию.

**Справочная информация**
- https://docs.python.org/3/library/re.html
- https://regex101.com
- https://en.wikibooks.org/wiki/Vehicle_Identification_Numbers_(VIN_codes)#Table_of_Contents
- https://en.wikibooks.org/wiki/Vehicle_Identification_Numbers_(VIN_codes)/World_Manufacturer_Identifier_(WMI)
- https://en.wikibooks.org/wiki/Vehicle_Identification_Numbers_(VIN_codes)/Printable_version#Model#
- https://en.wikipedia.org/wiki/List_of_Ford_factories

**Форматы VIN-кодов**

![варианты vin кодов](data/photo_5269222301897973406_x.jpg)

Настройка среды

In [4]:
# !pip freeze > requirements.txt

In [1]:
import re
import pandas as pd

Установка глобальных переменных

In [2]:
RANDOM_STATE = 42
DATA_PATH = 'data/vin_ford_train.txt'

## Знакомство с данными

In [3]:
with open (DATA_PATH, 'r') as f:
    data = f.read()
print(data[:400])

[2FMDK3JC4BBA41556:12500]мой[3FA6P0H75ER208976:14500]дядя [3FAHP08Z17R268380:1300][1FMCU0GX3DUC59421:15200]самых [1FAHP2E89DG119368:11600] верных[1FM5K8D82DGA16143:26400][1FTFW1CFXCFC23663:14000][2FMDK3JC6CBA90002:19500][1FTFW1CT8DKD06233:24400][1FMZU64W13UC28949:2900][2FMDK3JC9DBB30736:23500][1FMCU9D76CKC49193:12700][1FMCU9EG2CKA90624:20700][1FMCU0JX7EUA28208:21300][2FMGK5D85EBD31213:26700][2FMDK


### Выводы и наблюдения
- исходя из условий задачи, необходимый вариант VIN-кодов - `American VIN format`;
- в предоставленном файле VIN-коды находятся в неудобном для исследования формате:
  - необходима очистка и преобразование;
- помимо VIN-кодов в файле предоставлены цены.

## Подготовка паттерна
В соответствии с правилами составления VIN-кодов для американских автомобилей:
- современные VIN-коды состоят из 17 символов, которые не содержат букв I, O или Q;
- `check digit` - 0-9 or X;
- буквы U и Z и цифра 0 не используются для кода года;
- позиции 13–17 должны быть цифрами, тогда как позиция 12 может быть буквой или цифрой

### Разрешённые в VIN-кодах символы

In [4]:
allowed = r'ABCDEFGHJKLMNPRSTUVWXYZ1234567890'
allowed

'ABCDEFGHJKLMNPRSTUVWXYZ1234567890'

### Тестовый образец

In [5]:
vin_sample = '1FT7X2B6XCEB34867 1FTNF1CF4EKD18628 1GNKRFED3CJ319802 5N1BA08D49N601792 5TFHY5F19CX236781 JN1CV6AR7BM356524 ZFF65LJA9A0170669'
vin_sample

'1FT7X2B6XCEB34867 1FTNF1CF4EKD18628 1GNKRFED3CJ319802 5N1BA08D49N601792 5TFHY5F19CX236781 JN1CV6AR7BM356524 ZFF65LJA9A0170669'

### Северо-Американский формат VIN

In [13]:
wmi_pattern = r'(?P<wmi>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{3})'  # pos. 1-3
brake_pattern = r'(?P<brake>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890])' # pos. 4
body_pattern = r'(?P<body>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{3})'  # pos. 5-7
engine_pattern = r'(?P<engine>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890])'  # pos. 8
check_digit_pattern = r'(?P<check_digit>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890])'  # pos. 9
year_pattern = r'(?P<year>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890])'.replace('U', '').replace('Z', '').replace('0', '')  # pos. 10
plant_pattern = r'(?P<plant>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890])'  # pos. 11
snum_pattern = r'(?P<snum>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]\d{5})'  # pos. 12-17
pattern = f'{wmi_pattern}{brake_pattern}{body_pattern}{engine_pattern}{check_digit_pattern}{year_pattern}{plant_pattern}{snum_pattern}'
pd.DataFrame([x.groupdict() for x in re.compile(pattern).finditer(vin_sample)])

,wmi,brake,body,engine,check_digit,year,plant,snum
0,1FT,7,X2B,6,X,C,E,B34867
1,1FT,N,F1C,F,4,E,K,D18628
2,1GN,K,RFE,D,3,C,J,319802
3,5N1,B,A08,D,4,9,N,601792
4,5TF,H,Y5F,1,9,C,X,236781
5,JN1,C,V6A,R,7,B,M,356524
6,ZFF,6,5LJ,A,9,A,0,170669


### Извлечение производителя из VIN-кода

In [18]:
pattern = r'''
    \b(?P<wmi>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{3})  # три первые элемента слова, принадлежащие такому набору
    [ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{14}\b          # исключение 14-ти последующих элементов слова, принадлежащих набору
'''
re.compile(pattern, re.X).findall(vin_sample)

['1FT', '1FT', '1GN', '5N1', '5TF', 'JN1', 'ZFF']

### Поиск всех VIN-кодов определенной марки автомобиля на примере Ferrari
- Вин-коды Феррари ZDF, ZFF, ZSG

In [21]:
ferrari_wmi = ['ZDF', 'ZFF', 'ZSG']
wmi_pattern = '|'.join(ferrari_wmi)
pattern = re.compile(f'\\b(?:{wmi_pattern})[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{{14}}\\b')
pattern.findall(vin_sample)

['ZFF65LJA9A0170669']

### Поиск VIN-кодов с определенным регионом
Северная Америка как регион (США, Канада, Мексика)

    1, 4, 5 = United States
    2 = Canada
    3 = Mexico
    7F-70 = United States

Североамериканские машины

In [24]:
pattern = r'''
    \b(?:1|4|5|7F|70)[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{16}\b
    |\b2[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{16}\b
    |\b3[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{16}\b
    '''
re.compile(pattern, re.X).findall(vin_sample)

['1FT7X2B6XCEB34867',
 '1FTNF1CF4EKD18628',
 '1GNKRFED3CJ319802',
 '5N1BA08D49N601792',
 '5TFHY5F19CX236781']

Машины из любых других регионов (не Сев.Америка)

In [25]:
pattern = re.compile(r'\b(?!(?:1|4|5|7F|70|2|3)[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{16})[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{17}\b')
pattern.findall(vin_sample)

['JN1CV6AR7BM356524', 'ZFF65LJA9A0170669']

### Извлечение года из VIN-кода только североамериканских машин
- В североамериканском вине год 10й знак
- в кодировке года не используются знаки U и Z в дополнение к запрещенным I, O, Q

In [37]:
pattern = r'''
    \b(?:1|4|5|7F|70)[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{16}\b
    |\b2[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{16}\b
    |\b3[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{16}\b
'''
[x[9] for x in re.compile(pattern, re.X).findall(vin_sample)]

['C', 'E', 'C', '9', 'C']

Для проверки выведем весь датафрейм компонентов VIN кода

In [38]:
wmi_pattern = r'(?P<wmi>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{3})'  # pos. 1-3
brake_pattern = r'(?P<brake>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890])' # pos. 4
body_pattern = r'(?P<body>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{3})'  # pos. 5-7
engine_pattern = r'(?P<engine>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890])'  # pos. 8
check_digit_pattern = r'(?P<check_digit>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890])'  # pos. 9
year_pattern = r'(?P<year>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890])'.replace('U', '').replace('Z', '').replace('0', '')  # pos. 10
plant_pattern = r'(?P<plant>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890])'  # pos. 11
snum_pattern = r'(?P<snum>[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]\d{5})'  # pos. 12-17
pattern = f'{wmi_pattern}{brake_pattern}{body_pattern}{engine_pattern}{check_digit_pattern}{year_pattern}{plant_pattern}{snum_pattern}'
pd.DataFrame([x.groupdict() for x in re.compile(pattern).finditer(vin_sample)])

,wmi,brake,body,engine,check_digit,year,plant,snum
0,1FT,7,X2B,6,X,C,E,B34867
1,1FT,N,F1C,F,4,E,K,D18628
2,1GN,K,RFE,D,3,C,J,319802
3,5N1,B,A08,D,4,9,N,601792
4,5TF,H,Y5F,1,9,C,X,236781
5,JN1,C,V6A,R,7,B,M,356524
6,ZFF,6,5LJ,A,9,A,0,170669


### Проверка корректности формата VIN-кода

- Вин-код состоит из 17 знаков – цифр и букв, кроме 'I', 'O', 'Q'
- Последние три знака – всегда цифры

Создадим набор винов, где только один вин валидный и остальные ошибочные
- отрежем один знак
- добавим лишний знак
- заменим буквой последнюю цифру в номере
- заменим разрешенный знак на неразрешенный

In [39]:
error_sample = vin_sample + ' 3LN6L2GKXDR811732 BAAM3334XKC59086 1N4BL11D84C1859519 1C4RJEBG5EC48287Q QN1AB7AP5EL654699'
error_sample

'1FT7X2B6XCEB34867 1FTNF1CF4EKD18628 1GNKRFED3CJ319802 5N1BA08D49N601792 5TFHY5F19CX236781 JN1CV6AR7BM356524 ZFF65LJA9A0170669 3LN6L2GKXDR811732 BAAM3334XKC59086 1N4BL11D84C1859519 1C4RJEBG5EC48287Q QN1AB7AP5EL654699'

Валидные VIN-коды

In [48]:
pattern = re.compile(r'\b[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{14}\d{3}\b')
pattern.findall(error_sample)

['1FT7X2B6XCEB34867',
 '1FTNF1CF4EKD18628',
 '1GNKRFED3CJ319802',
 '5N1BA08D49N601792',
 '5TFHY5F19CX236781',
 'JN1CV6AR7BM356524',
 'ZFF65LJA9A0170669',
 '3LN6L2GKXDR811732']

Не валидные VIN-коды

In [52]:
pattern = re.compile(r'(?!\b[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{14}\d{3}\b)\b\w+\b')

pattern.findall(error_sample)

['BAAM3334XKC59086',
 '1N4BL11D84C1859519',
 '1C4RJEBG5EC48287Q',
 'QN1AB7AP5EL654699']

### Автомобили Ford

In [53]:
ford_wmi = {
    'AFA': 'Ford Motor Company of Southern Africa & Samcor',
    'AFB': 'Mazda BT-50 made by Ford Motor Company of Southern Africa',
    'JC0': 'Ford brand cars made by Mazda',
    'JC2': 'Ford Courier made by Mazda',
    'KNJ': 'Ford Festiva & Aspire made by Kia',
    'LJX': 'JMC Ford',
    'LVS': 'Changan Ford & Changan Ford Mazda',
    'MAJ': 'Ford India',
    'MNB': 'Ford Thailand',
    'NM0': 'Ford Otosan',
    'PE1': 'Ford Motor Company Philippines',
    'PE3': 'Mazda Philippines made by Ford Motor Company Philippines',
    'PR8': 'Ford',
    'LFA': 'Ford Lio Ho Motor Co Ltd. old designation',
    'RHA': 'Ford Lio Ho Motor Co Ltd. new designation',
    'RL0': 'Ford Vietnam',
    'SBC': 'Iveco Ford Truck',
    'SFA': 'Ford UK',
    'VSK': 'Nissan Motor Iberica SA, Nissan passenger car/MPV/van/SUV/pickup & Ford Maverick 1993–1999',
    'VS6': 'Ford Spain',
    'WF0': 'Ford Germany',
    'X9F': 'Ford Motor Company ZAO',
    'Z6F': 'Ford Sollers (Russia)',
    '1FA': 'Ford car',
    '1FB': 'Ford "bus" (van with more than 3 rows of seats)',
    '1FC': 'Ford stripped chassis made by Ford',
    '1FD': 'Ford incomplete vehicle',
    '1FM': 'Ford MPV/SUV',
    '1FT': 'Ford truck',
    '1F1': 'Ford SUV - Limousine (through 2009)',
    '1F6': 'Ford stripped chassis made by Detroit Chassis LLC',
    '1ZV': 'Ford made by AutoAlliance International',
    '2FA': 'Ford car',
    '2FM': 'Ford MPV/SUV',
    '2FT': 'Ford truck',
    '3FA': 'Ford car',
    '3FC': 'Ford stripped chassis made by Ford & IMMSA',
    '3FE': 'Ford Mexico',
    '3FM': 'Ford MPV/SUV',
    '3FN': 'Ford F-650/F-750 made by Blue Diamond Truck Co. (truck)',
    '3FR': 'Ford F-650/F-750 made by Blue Diamond Truck Co. (incomplete vehicle)',
    '3FT': 'Ford truck',
    '4F2': 'Mazda SUV made by Ford',
    '4F4': 'Mazda truck made by Ford',
    '4N2': 'Nissan Quest made by Ford',
    '5LD': 'Ford & Lincoln incomplete vehicle – limousine (2010–2014)',
    '6F1': 'Ford',
    '6FP': 'Ford Australia',
    '7A5': 'Ford New Zealand',
    '8AF': 'Ford Argentina',
    '9BF': 'Ford Brazil',
}

In [55]:
ford_groups = {
    'P': 'All Ford, Mercury, Merkur, and Lincoln passenger cars 1981-1986, Ford-brand passenger cars made in North America at a factory owned 100% by Ford Motor Co. 1987-2009',
    'M': 'Mercury & Lincoln passenger cars made in North America at a factory owned 100% by Ford Motor Co. 1987-2009. Mercury passenger cars 2010-2011',
    'T': 'Ford, Mercury, and Merkur passenger cars made outside North America or made in North America at a factory not owned 100% by Ford Motor Co. 1987-2009 (Ford Festiva, Aspire, Probe, & \'05-\'09 Mustang, Mercury Capri \'91-\'94, & Cougar \'99-\'02, Merkur XR4Ti \'87-\'89 and Scorpio \'88-\'89)',
    'P': 'Ford-brand passenger cars 2010-',
    'L': 'Lincoln passenger cars 2010-2020',
}

Отбор автомобилей FORD

In [58]:
ford_wmi_pattern = '|'.join(ford_wmi.keys())
pattern = f'\\b(?:{ford_wmi_pattern})[ABCDEFGHJKLMNPRSTUVWXYZ1234567890]{{14}}\\b'
re.compile(pattern).findall(vin_sample)

['1FT7X2B6XCEB34867', '1FTNF1CF4EKD18628']